In [62]:
# Instalações necessárias das apis do google para rodar este projeto

%pip -q install google-genai
!pip install -q google-adk

In [63]:
# Configurar a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [64]:
# Configurar o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [65]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [66]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [67]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [82]:
##########################################
# --- Agente 1: Revisor de dados do usuário --- #
##########################################

def agente_revisor_dados(dados):

    revisor_dados = Agent(
        name="agente_revisor_dados",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente responsável por revisar os dados digitados pelo usuário.
        Caso o usuário informe um número irreal de dias por semana para corrida (ex: mais de 7 ou menos de 1),
        ou um tempo exagerado por dia (ex: mais de 5 horas), você deve substituir por valores genéricos:
        3 dias por semana e 1 hora de treino por dia
        Além disso, se algum dado estiver ausente ou mal formatado, insira o valor padrão
        e retorne uma mensagem de aviso sobre a correção.
        """,
        description="Agente que revisa os dados do usuário",
    )

    entrada_do_agente_revisor_dados = f"Dados do usuário: {dados}"

    dados_revisados = call_agent(revisor_dados, entrada_do_agente_revisor_dados)
    return dados_revisados

In [83]:
##########################################
# --- Agente 2: Buscador de Treinos de Corrida --- #
##########################################

def agente_buscador(treino, dados_revisados):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente especializado em corrida, responsável por pesquisar treinos eficientes de acordo com o perfil do usuário.
        Use a ferramenta de busca do Google (google_search) para encontrar sugestões atualizadas e eficazes de treinos de corrida.
        Sempre cumprimente o usuário usando o nome fornecido, mantendo um tom amigável e acolhedor.
        Forneça as informações de forma objetiva e clara — sem exagerar nos detalhes, mas garantindo que o usuário saiba como começar.
        Inclua dicas personalizadas conforme os dados revisados do usuário, considerando o objetivo dele com a corrida.
        Evite repetir informações, cite fontes relevantes e use uma linguagem encorajadora.
        """,
        description="Agente que gera informações sobre corrida",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Treino: {treino}, Dados Revisados: {dados_revisados}"

    treinos_pesquisados = call_agent(buscador, entrada_do_agente_buscador)
    return treinos_pesquisados

In [84]:
################################################
# --- Agente 3: Planejador de treinos --- #
################################################

def agente_planejador(treinos_pesquisados, treinos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de treinos especialista em corrida.
        Com base na lista de treinos fornecida pelo agente buscador, seu papel é organizar e criar um plano de treino personalizado para o usuário.
        Explique detalhadamente o que é cada treino, seus benefícios e para que serve.
        No final, selecione os treinos mais relevantes para o objetivo do usuário e retorne esse plano de forma clara e objetiva.
        Caso precise, você pode usar a ferramenta de busca do Google (google_search) para complementar informações específicas.
        Mantenha um tom encorajador e amigável.
        """,
        description="Agente que planeja treinos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Treino: {treinos_pesquisados}, Treinos buscados: {treinos_buscados}"
    # Executa o agente
    plano_do_treino = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_treino

In [85]:
######################################
# --- Agente 4: Treinador de Corrida --- #
######################################

def agente_treinador(plano_do_treino, plano_de_treino):
    treinador = Agent(
        name="agente_treinador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um treinador especializado em treinos de corrida para todos os níveis: iniciantes, intermediários e avançados.
        Utilize os treinos mais relevantes do plano fornecido para criar uma planilha rascunhada de 4 semanas de treinos.
        A planilha deve ser organizada com colunas representando as semanas (Semana 1, Semana 2, Semana 3, Semana 4)
        e linhas representando os dias da semana (segunda, terça, quarta, quinta, sexta, sábado e domingo).
        Personalize o plano conforme as necessidades e nível do usuário, garantindo que os treinos sejam adequados e progressivos.
        Obs: Se algum dia da semana não tiver treino previsto, escreva explicitamente ‘Descanso’. Nunca deixe campos em branco.
        Use um tom motivacional e encorajador para ajudar o usuário a se sentir confiante para seguir o plano.
        """,
        description="Agente especializado em organizar treinos de corrida"
    )
    entrada_do_agente_treinador = f"Treinos: {plano_do_treino}, Plano de treino: {plano_de_treino}"

    rascunho = call_agent(treinador, entrada_do_agente_treinador)
    return rascunho

In [86]:
##########################################
# --- Agente 5: Revisor de Treinos --- #
##########################################

def agente_revisor_planilha(rascunho, planilha_gerada):
    revisor = Agent(
        name="agente_revisor_planilha",
        model="gemini-2.0-flash",
        instruction="""
          Você é um revisor de planilhas de treino especializado em corrida.
          Revise o conteúdo da planilha abaixo observando os seguintes aspectos:
          Clareza (as instruções estão fáceis de entender?)
          Concisão (sem repetições desnecessárias?)
          Correção (há erros ou inconsistências?)
          Tom (o plano motiva e respeita o nível do usuário?)
          Obs: Não considere o uso da palavra "descanso" como repetição desnecessária, ela é
          essencial na planilha para indicar dias de recuperação e deve aparecer sempre que necessário.
          Se tudo estiver ótimo, responda apenas: 'A planilha está ótima e pronta para usar!'.
          Caso contrário, liste os pontos que precisam ser melhorados e dê sugestões claras.
            """,
        description="Agente revisor de treinos para corrida."
    )
    entrada_do_agente_revisor = f"Treinos: {rascunho}\nPlanilha: {planilha_gerada}"

    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [87]:
#############################################
# --- Agente 6: Finalizador de Planilha --- #
#############################################

def agente_finalizador(treinos, planilha_original, revisao):
    finalizador = Agent(
        name="agente_finalizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em finalizar planilhas de treino de corrida.
        Sua tarefa é revisar a planilha original com base na resposta do revisor.
        Se a revisão disser que está tudo certo (ex: "A planilha está ótima e pronta para usar!"), apenas retorne a planilha original.
        Se houver sugestões de melhoria, você deve aplicar essas sugestões na planilha, mantendo o formato original:
        Colunas: Semana 1, Semana 2, Semana 3, Semana 4
        Linhas: Segunda, Terça, Quarta, Quinta, Sexta, Sábado, Domingo
        A nova versão da planilha deve ser clara, organizada e adequada ao perfil do usuário.
        """,
        description="Agente responsável por aplicar revisões e finalizar a planilha de treinos."
    )
    entrada_do_agente_finalizador = f"Treinos originais: {treinos}, Planilha original: {planilha_original}, Revisão: {revisao}"

    planilha_final = call_agent(finalizador, entrada_do_agente_finalizador)
    return planilha_final

In [88]:
print(" 🏁 Bem-vindo ao seu Treinador de Corridas Personalizado! 🏃‍♀️")

# --- Obter dados do usuário ---

print("\nPor favor, informe os dados abaixo: \n")

nome = input("Qual é o seu nome? ")
while not nome:
  nome = input("Por favor, informe um nome válido.\nQual é o seu nome? ")


idade = input("\nQual a sua idade? ")
while not idade:
  idade = input("Por favor, informe uma idade válida.\nQual é o sua idade? ")


nivel = input("\nComo você descreveria sua experiência atual na corrida? (Iniciante, Intermediário, Avançado) ")
while not nivel:
  nivel = input("Por favor, informe sua experiencia na corrida: ")


semanas = int(input("\nEm quantos dias da semana você pretende treinar corrida? "))
while not semanas:
    semanas = int(input("Por favor, quantos dias da semana você pretende treinar corrida: "))


dia = input("\nEm quais dias da semana você geralmente pode treinar? (Ex: Seg, Qua, Sex) ")
while not dia:
    dia = input("Por favor, informe os dias da semana que você pode treinar: ")


tempo = input("\nEm média, quanto tempo você dedica a cada treino de corrida? ")
while not tempo:
    tempo = input("Por favor, informe quanto tempo você dedica a cada sessão de corrida: ")


objetivo = input("\nQual é o seu principal objetivo com a corrida atualmente? (Ex: Correr uma prova de 5km, melhorar meu tempo em 10km, sentir mais disposição, etc.) ")
while not objetivo:
    objetivo = input("Por favor, informe seu objetivo com a corrida: ")


observacao = input("\nGostaria de adicionar alguma informação relevante sobre sua saúde, histórico de lesões ou outras considerações para o seu treino? ")


# --- Armazenar os dados do usuário em apenas uma variável ---
user_message = f"""
Nome: {nome}
Idade: {idade}
Nivel: {nivel}
Dias de treino por semana: {semanas}
Tempo de treino por dia: {tempo}
Dia da semana: {dia}
Objetivo: {objetivo}
Observação: {observacao}
"""

# Inserir lógica do sistema de agentes ################################################

print("\nCerto! Criaremos seu plano de corrida personalizado! 💪")

revisor_dados = agente_revisor_dados(user_message)
print("\n--- ✅ Resultado do revisor de dados ---\n")
display(to_markdown(revisor_dados))
print("--------------------------------------------------------------")

treinos_buscados = agente_buscador(user_message, revisor_dados)
print("\n--- ✅ Resultado da busca de treinos  ---\n")
display(to_markdown(treinos_buscados))
print("--------------------------------------------------------------")

plano_de_treino = agente_planejador(user_message, treinos_buscados)
print("\n--- ✅ Resultado do planejador de treinos ---\n")
display(to_markdown(plano_de_treino))
print("--------------------------------------------------------------")

rascunho_de_treino = agente_treinador(user_message, plano_de_treino)
print("\n--- ✅ Resultado do rascunho de treinos ---\n")
display(to_markdown(rascunho_de_treino))
print("--------------------------------------------------------------")

revisor = agente_revisor_planilha(user_message, rascunho_de_treino)
print("\n--- ✅ Resultado do revisor de treinos ---\n")
display(to_markdown(revisor))
print("--------------------------------------------------------------")

planilha_final = agente_finalizador(user_message, rascunho_de_treino, revisor)
print("\n--- ✅ Resultado da planilha final ---\n")
print("\n🎉 Seu plano de corrida personalizado está pronto! Consulte abaixo:")
display(to_markdown(planilha_final))
print("\nBoa corrida! 😄")
print("--------------------------------------------------------------")

 🏁 Bem-vindo ao seu Treinador de Corridas Personalizado! 🏃‍♀️

Por favor, informe os dados abaixo: 

Qual é o seu nome? Clara

Qual a sua idade? 21 anos

Como você descreveria sua experiência atual na corrida? (Iniciante, Intermediário, Avançado) Avançado

Em quantos dias da semana você pretende treinar corrida? 3

Em quais dias da semana você geralmente pode treinar? (Ex: Seg, Qua, Sex) Terça, quinta e sabado

Em média, quanto tempo você dedica a cada treino de corrida? 1 hora

Qual é o seu principal objetivo com a corrida atualmente? (Ex: Correr uma prova de 5km, melhorar meu tempo em 10km, sentir mais disposição, etc.) correr 5km em menos de 25 min

Gostaria de adicionar alguma informação relevante sobre sua saúde, histórico de lesões ou outras considerações para o seu treino? Nenhuma

Certo! Criaremos seu plano de corrida personalizado! 💪

--- ✅ Resultado do revisor de dados ---



> Os dados do usuário estão completos e bem formatados, sem necessidade de alterações.
> 


--------------------------------------------------------------

--- ✅ Resultado da busca de treinos  ---



> Olá, Clara! Tudo bem? Que bom ter você por aqui!
> 
> Com base no seu perfil de corredora avançada de 21 anos, que treina 3 vezes por semana com o objetivo de correr 5km em menos de 25 minutos, preparei algumas sugestões de treinos que podem te ajudar a alcançar essa meta.
> 
> 
> Para te ajudar a atingir seu objetivo de correr 5 km em menos de 25 minutos, preparei um plano de treino focado em velocidade e resistência, combinando diferentes tipos de exercícios para otimizar seu desempenho.
> 
> ### Sugestões de Treino:
> 
> 1.  **Treino Intervalado:**
> 
> *   **Terça-feira:** Treino intervalado de alta intensidade (HIIT).
>     *   Aquecimento: 10 minutos de trote leve.
>     *   Série principal: 8-12 repetições de 1 minuto de corrida intensa (próximo ao seu ritmo de prova) seguidos por 30 segundos de descanso ativo (trote leve ou caminhada).
>     *   Volta à calma: 5 minutos de trote leve.
>     *   **Benefícios:** Aumenta a capacidade cardiorrespiratória e a velocidade.
> 2.  **Treino de Ritmo (Tempo Run):**
> 
> *   **Quinta-feira:** Treino de ritmo para melhorar a resistência.
>     *   Aquecimento: 10 minutos de trote leve.
>     *   Série principal: 20-25 minutos de corrida em ritmo constante e desafiador, ligeiramente mais lento que seu ritmo de prova de 5 km.
>     *   Volta à calma: 5 minutos de trote leve.
>     *   **Benefícios:** Melhora a resistência muscular e a capacidade de manter um ritmo rápido por mais tempo.
> 3.  **Treino Longo Leve:**
> 
> *   **Sábado:** Corrida leve e contínua.
>     *   Aquecimento: 5 minutos de trote leve.
>     *   Série principal: 35-40 minutos de corrida em ritmo confortável e conversacional.
>     *   Volta à calma: 5 minutos de caminhada leve.
>     *   **Benefícios:** Aumenta a resistência aeróbica e a capacidade do corpo de usar gordura como fonte de energia.
> 
> ### Dicas Adicionais:
> 
> *   **Aquecimento:** Antes de cada treino, faça um aquecimento adequado com alongamentos dinâmicos e um trote leve.
> *   **Descanso:** Priorize o descanso e a recuperação muscular, especialmente após os treinos mais intensos.
> *   **Alimentação:** Mantenha uma dieta equilibrada e rica em nutrientes para otimizar o desempenho e a recuperação.
> *   **Fortalecimento:** Inclua exercícios de fortalecimento muscular (como musculação ou treinamento funcional) em sua rotina para prevenir lesões e melhorar a eficiência da corrida.
> *   **Técnica de Corrida:** Trabalhe na sua técnica de corrida para otimizar a biomecânica e reduzir o risco de lesões.
> 
> Lembre-se que a consistência é fundamental para alcançar seus objetivos. Mantenha-se motivada, ouça seu corpo e ajuste o plano de treino conforme necessário. Com dedicação e disciplina, você certamente alcançará sua meta de correr 5 km em menos de 25 minutos. 😉
> 


--------------------------------------------------------------

--- ✅ Resultado do planejador de treinos ---



...

--------------------------------------------------------------

--- ✅ Resultado do revisor de treinos ---



> A planilha está ótima e pronta para usar!
> 


--------------------------------------------------------------

--- ✅ Resultado da planilha final ---


🎉 Seu plano de corrida personalizado está pronto! Consulte abaixo:


> Olá, Clara! 😊
> 
> Com base no plano de treino que preparei para você, aqui está um rascunho da sua planilha de 4 semanas. Lembre-se que este é um plano flexível, e você pode ajustá-lo conforme necessário, de acordo com seu corpo e sua rotina. O mais importante é manter a consistência e a motivação!
> 
> **Planilha de Treino de 4 Semanas**
> 
> | Dia da Semana | Semana 1                                                                                                   | Semana 2                                                                                                   | Semana 3                                                                                                   | Semana 4                                                                                                     |
> | :------------ | :--------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------- |
> | **Segunda**    | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                     |
> | **Terça**     | Treino Intervalado (8 repetições de 1 minuto intenso com 30 segundos de descanso)                           | Treino Intervalado (10 repetições de 1 minuto intenso com 30 segundos de descanso)                          | Treino Intervalado (12 repetições de 1 minuto intenso com 30 segundos de descanso)                          | Treino Intervalado (12 repetições de 1 minuto intenso com 30 segundos de descanso)                          |
> | **Quarta**    | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                     |
> | **Quinta**    | Treino de Ritmo (20 minutos em ritmo constante e desafiador)                                               | Treino de Ritmo (22 minutos em ritmo constante e desafiador)                                               | Treino de Ritmo (25 minutos em ritmo constante e desafiador)                                               | Treino de Ritmo (25 minutos em ritmo constante e desafiador)                                               |
> | **Sexta**     | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                     |
> | **Sábado**    | Treino Longo Leve (35 minutos em ritmo confortável)                                                          | Treino Longo Leve (38 minutos em ritmo confortável)                                                          | Treino Longo Leve (40 minutos em ritmo confortável)                                                          | Treino Longo Leve (40 minutos em ritmo confortável)                                                          |
> | **Domingo**   | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                   | Descanso                                                                                                     |
> 
> **Observações:**
> 
> *   **Aquecimento e Volta à Calma:** Não se esqueça de incluir 10 minutos de aquecimento e 5 minutos de volta à calma em todos os treinos.
> *   **Intensidade:** Ajuste a intensidade dos treinos intervalados e de ritmo com base em como você se sente. O objetivo é desafiar-se, mas sem exagerar!
> *   **Descanso:** O descanso é crucial para a recuperação e para evitar lesões. Respeite os dias de descanso e ouça o seu corpo.
> *   **Progressão:** A progressão é leve e gradual. Aumentar o tempo ou a intensidade muito rapidamente pode levar a lesões.
> 
> Lembre-se de que estou aqui para te ajudar em cada passo do caminho. Se tiver alguma dúvida ou precisar de ajustes no plano, é só me avisar! 😉
> 



Boa corrida! 😄
--------------------------------------------------------------
